In [12]:
#Recurrent Neural Networks
from __future__ import print_function

import tensorflow as tf 
from tensorflow.contrib import rnn
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/MNIST_data", one_hot = True)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

Extracting data/MNIST_data\train-images-idx3-ubyte.gz
Extracting data/MNIST_data\train-labels-idx1-ubyte.gz
Extracting data/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting data/MNIST_data\t10k-labels-idx1-ubyte.gz


In [13]:
n_input = 28
n_steps = 28
n_hidden = 128
n_classes = 10

learning_rate = 0.001
batch_size = 128
display_step = 200
training_steps = 10000


#TF Graph oluşturma 
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

weights = {
    'out':tf.Variable(tf.random_normal([n_hidden, n_classes]))
}

biases = {
    'out':tf.Variable(tf.random_normal([n_classes]))
}

In [14]:
def RNN(x, weights, biases):
    
    x = tf.unstack(x, n_steps, 1)
    
    #Bir LSTM hücresini tensorflow ile tanımlama
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias = 1.0)
    
    #LSTM çıktısını elde etme
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    
    #Lineer aktivasyon fonksiyonu
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

#loss ve optimizer tanımlama

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = pred, labels = y))

optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

#Modeli değerlendirme

ValueError: Variable rnn/basic_lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Users\ITU\anaconda\envs\tf\lib\site-packages\tensorflow_core\python\framework\ops.py", line 1748, in __init__
    self._traceback = tf_stack.extract_stack()
  File "C:\Users\ITU\anaconda\envs\tf\lib\site-packages\tensorflow_core\python\framework\ops.py", line 3426, in _create_op_internal
    op_def=op_def)
  File "C:\Users\ITU\anaconda\envs\tf\lib\site-packages\tensorflow_core\python\framework\ops.py", line 3357, in create_op
    attrs, op_def, compute_device)
  File "C:\Users\ITU\anaconda\envs\tf\lib\site-packages\tensorflow_core\python\util\deprecation.py", line 507, in new_func
    return func(*args, **kwargs)
  File "C:\Users\ITU\anaconda\envs\tf\lib\site-packages\tensorflow_core\python\framework\op_def_library.py", line 794, in _apply_op_helper
    op_def=op_def)


In [16]:
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

#Eğitime başlama

with tf.Session() as sess:
    sess.run(init)
    step = 1
    
    #max iterasyon sayısına ulaşana kadar eğitimi sürdür
    while step * batch_size < training_steps:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        
        if step % display_step == 0:
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            
            print("Iter " + str(step*batch_size) + ", Minibatch Loss = " + "{:.6f}".format(loss) + ", Training Accuracy = " + \
                  "{:.5f}".format(acc))
            
        step += 1
    print("Optimization Finished!")
    
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    
    print("Testing accuracy:", sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

NameError: name 'optimizer' is not defined